# Семинар 9: Character-Level LSTM

## Вступление
На прошлом занятии мы познакомились с тем, как можно векторизовать текстовые данные для решения задач обработки текстов. Сегодня мы продолжим заниматься текстами и посмотрим на простейший пример автоматической генерации текстов при помощи Recurrent Neural Network (RNN).

Полезные материалы по RNN можно почитать [здесь](http://karpathy.github.io/2015/05/21/rnn-effectiveness/), а реализацию на PyTorch — [здесь](https://github.com/karpathy/char-rnn).

### План семинара
1. Подготовка данных
2. Имплементация модели
3. Обучение модели
4. Применение модели

https://github.com/hse-ds/iad-deep-learning/blob/master/2023/seminars/09.%20LSTM/09_Character_Level_LSTM_solved.ipynb

In [ ]:
!pip install lightning -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import warnings
from typing import Iterable, Tuple

import pytorch_lightning as pl
import torch
from torch import nn

warnings.filterwarnings("ignore")

## 1. Подготовка данных

### Загрузим текст "Анны Карениной"

In [ ]:
!wget https://raw.githubusercontent.com/hse-ds/iad-deep-learning/master/2022/seminars/sem09/anna.txt

--2025-03-01 17:27:26--  https://raw.githubusercontent.com/hse-ds/iad-deep-learning/master/2022/seminars/sem09/anna.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1985223 (1.9M) [text/plain]
Saving to: ‘anna.txt.1’

anna.txt.1          100%[===================>]   1.89M  --.-KB/s    in 0.05s   

2025-03-01 17:27:26 (38.5 MB/s) - ‘anna.txt.1’ saved [1985223/1985223]



In [ ]:
with open("anna.txt", "r") as f:
    text = f.read()

text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

В тексте есть пробелы, переносы строки - это важно.

### Токенизируем текст

Аналогично предыдущему семинару, в ячейках ниже создадим два словаря для преобразования символов в целые числа и обратно.

In [ ]:
unique_chars = tuple(set(text))
int2char = dict(enumerate(unique_chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = torch.tensor([char2int[ch] for ch in text])
encoded[:100]

tensor([44, 27, 23,  0, 70, 24,  5, 17, 16, 33, 33, 33, 35, 23,  0,  0, 54, 17,
        25, 23, 57, 74, 64, 74, 24,  8, 17, 23,  5, 24, 17, 23, 64, 64, 17, 23,
        64, 74, 39, 24, 47, 17, 24, 58, 24,  5, 54, 17, 55, 11, 27, 23,  0,  0,
        54, 17, 25, 23, 57, 74, 64, 54, 17, 74,  8, 17, 55, 11, 27, 23,  0,  0,
        54, 17, 74, 11, 17, 74, 70,  8, 17, 56, 26, 11, 33, 26, 23, 54, 82, 33,
        33, 62, 58, 24,  5, 54, 70, 27, 74, 11])

Здесь не важно частота слова, здесь важны буквы, не важен порядок чисел.

Первые 100 символов.

Посмотрим на схему char-RNN:
<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charseq.jpeg?raw=1" width="30%">

Сеть ожидает **one-hot encoded** входа, что означает, что каждый символ преобразуется в целое число (через созданный маппинг), а затем преобразуется в вектор-столбец, где только соответствующий ему целочисленный индекс будет иметь значение 1, а остальная часть вектора будет заполнена нулями. Давайте напишем функцию для этого преобразования.

#### Задание: допишите функцию one-hot кодирования последовательности

В hidden layer у нас LSTM, в предсказаниях - Softmax.

LSTM - сохраняет состояние(информацию) из предыдущих слоев. Он долгое время запоминает контекст.

Переводим наш текст в вектора, дальше напишем модель и потренируем ее.

In [ ]:
def one_hot_encode(int_words: torch.Tensor, n_labels: int) -> torch.Tensor:
    """
    Creates one-hot representation matrix for a given batch of integer sequences
    :param int_words: tensor of ints, which represents current sequence; shape: [batch_size, seq_len]
    :param n_labels: vocabulary size (number of unique tokens in data)
    :return: one-hot representation of the input tensor; shape: [batch_size, seq_len, n_labels]
    """
    words_one_hot = torch.zeros(
        (int_words.numel(), n_labels), dtype=torch.float32, device=int_words.device # формируем вектор: размер словаря умноженное сколько хотим лейблов. Пример хотим 8 лейблов, слов 3 - пространство 3 на 8
    ) # первый шаг - создам вектор с ноликами такого размера, второй шаг. Пример: есть сампл с тремя числами [1,3,5]. Длина словаря = 5. Будет три вектора для каждого сэмпла
    words_one_hot[torch.arange(words_one_hot.shape[0]), int_words.flatten()] = 1.0
    words_one_hot = words_one_hot.reshape((*int_words.shape, n_labels))

    return words_one_hot

In [ ]:
# testing the function
test_seq = torch.tensor([[3, 5, 1], [0, 2, 4]]) # хотим вектор длины 8, у нас два семпла, первый семпл - тензор 2 на 2
test_one_hot = one_hot_encode(test_seq, 8)

print(test_one_hot)
# есть индекс 3, у него стоит 1, индекс 5, у него на второй строке тоже 1, индекс 1 на третьей строке тоже 1
# 8 - длина словаря

tensor([[[0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.]]])


### Сформируем батчи
На простом примере батчи будут выглядеть так: мы возьмем закодированные символы и разделим их на несколько последовательностей, заданных параметром `batch_size`. Каждая из наших последовательностей будет иметь длину `seq_length`.

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/sequence_batching@1x.png?raw=1" width=500px>

**1. Отбросим часть текста, чтобы у нас были только полные батчи**

Каждый батч содержит $N \times M$ символов, где $N$ — это количество последовательностей в батче (`batch_size`), а $M$ — длина каждой последовательности (`seq_length`). Затем, чтобы получить общее количество батчей $K$, которое мы можем сделать из последовательности, нужно разделить длину последовательности на количество символов в батче. Когда мы узнаем количество батчей, можно получить общее количество символов, которые нужно сохранить, из последовательности: $N \times M \times K$.

**2. Разделим текст на $N$ частей**

Этот шаг нужен, чтобы мы могли проходить по тексту окном размера `[batch_size, seq_len]`. Его можно реализовать при помощи простого `reshape`.

**3. Теперь, когда у нас готова матрица текста, мы можем двигаться по ней окном, чтобы получить батчи**

Из каждой позиции окна сформируем обучающие пары `(x, y)` следующим образом: $x$ — это все элементы окна кроме последнего столбца, а $y$ — это все элементы окна кроме первого столбца. Тем самым для каждого токена исходного текста мы будем предсказывать следующий за ним токен.

#### Задание: допишите функцию генерации батчей

Простыми словами мы хотим, чтобы у нас модель видела 3 слова до и после, то есть последовательность слов была 3 до и 3 после.

Для этого нужно сформировать батчи поэлементно (пример с фильтром).
Далее нужно посмотреть, сколько батчей и последновательностей у нас внутри текста.
Мы можем узнать, сколько символов нам нужно.

То есть нужно поделить текст на батчи, внутри батчей пройтись sequences, набрать сэмплом (x,y).

In [ ]:
def get_batches(
    int_words: torch.Tensor, batch_size: int, seq_length: int
) -> Iterable[torch.Tensor]:
    """
    Generates batches from encoded sequence.
    :param int_words: tensor of ints, which represents the text; shape: [batch_size, -1]
    :param batch_size: number of sequences per batch
    :param seq_length: number of encoded chars in a sequence
    :return: generator of pairs (x, y); x_shape, y_shape: [batch_size, seq_length - 1]
    """
    # 1. Truncate text, so there are only full batches, нужно посчитать количество батчей
    window_size = seq_length + 1
    batch_size_total = batch_size * window_size # все количество данных для одного батча
    n_batches = len(int_words) // batch_size_total # длина всего текста поделить на то, сколько батчей будет
    int_words = int_words[: n_batches * batch_size_total] # убираем слова, которые не влезают

    # 2. Reshape into batch_size rows
    int_words = int_words.reshape((batch_size, -1)) #транспонируем вектора

    # 3. Iterate through the text matrix, делим данные на батчи
    for position in range(0, int_words.shape[1], window_size):
        x = int_words[:, position : position + window_size - 1]
        y = int_words[:, position + 1 : position + window_size]
        yield x, y

In [ ]:
# testing the function
test_batches = get_batches(encoded, 8, 50) #encoded - 100 первых слов в тексте анны корениной, размер батча = 8, длина последовательности=50
test_x, test_y = next(test_batches)
assert test_x.shape == test_y.shape
print(f"x:\n{test_x[:10, :10]}\n")
print(f"y:\n{test_y[:10, :10]}")
# в y сдвиг на 1 индекс

x:
tensor([[44, 27, 23,  0, 70, 24,  5, 17, 16, 33],
        [ 0,  0, 24, 76, 17, 23, 11, 76, 17,  8],
        [70, 27, 23, 70, 17, 74, 11,  8, 56, 64],
        [24, 26, 17, 70, 27, 56, 55, 48, 27, 70],
        [24,  5, 65, 17, 27, 24, 17, 27, 23, 76],
        [ 5, 65, 17, 26, 27, 56, 17, 26, 23,  8],
        [ 8, 70, 17, 29, 24, 17, 69, 56, 58, 24],
        [65, 17, 68, 64, 24,  1, 24, 54, 17, 68]])

y:
tensor([[27, 23,  0, 70, 24,  5, 17, 16, 33, 33],
        [ 0, 24, 76, 17, 23, 11, 76, 17,  8, 27],
        [27, 23, 70, 17, 74, 11,  8, 56, 64, 55],
        [26, 17, 70, 27, 56, 55, 48, 27, 70, 25],
        [ 5, 65, 17, 27, 24, 17, 27, 23, 76, 17],
        [65, 17, 26, 27, 56, 17, 26, 23,  8, 17],
        [70, 17, 29, 24, 17, 69, 56, 58, 24,  5],
        [17, 68, 64, 24,  1, 24, 54, 17, 68, 64]])


### Наконец, подготовим класс датасета

In [ ]:
class AnnaData(torch.utils.data.IterableDataset): # загружаем наш датасет
    def __init__(self, int_words: torch.Tensor, batch_size: int, seq_length: int):
        self.int_words = int_words # задаем количество слов
        self.batch_size = batch_size # размер батча
        self.seq_length = seq_length # последовательность

    def __iter__(self):
        return get_batches(self.int_words, self.batch_size, self.seq_length) # вызываем функцию get_batches

## 2. Имплементация модели

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charRNN.png?raw=1" width=50%>

### Структура модели

* Создаём и храним необходимые словари.
* Определяем слой [LSTM]((https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM)) с помощью инстанса класса `torch.nn.LSTM`, который принимает набор параметров: `input_size` — длина последовательности в батче; `n_hidden` — размер скрытых слоёв; `n_layers` — количество слоёв; `drop_prob` — вероятность дропаута; и `batch_first` — флаг, указывающий на то, что у входных последовательностей размерность батча идёт вдоль нулевой оси.
* Определяем слой Dropout с таким же значением `drop_prob`.
* Определяем полносвязный слой с набором параметров: размерность ввода — `n_hidden`; размерность выхода — размер словаря.
* Наконец, инициализируем веса и начальное скрытое состояние (`self.init_hidden()`).

В модель будет на вход по-одному символ, по этому символу модель должна будет предсказать следующий.

Вероятность следующего символа зависит не только от входного. LSTM - хранит контекст на предудущих шагах.



In [ ]:
class CharRNN(nn.Module): #LSTM уже написанная
    def __init__(
        self,
        unique_tokens: Tuple[str],
        n_hidden: int = 256,
        n_layers: int = 2,
        drop_prob: float = 0.5,
    ) -> None:
        super().__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.drop_prob = drop_prob

        # create mappings, преобразуем наши индексы в символы (предсказываем буквы)
        self.unique_tokens = unique_tokens
        self.int2char = dict(enumerate(self.unique_tokens))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()} # важно делать обратный словарь для читабельности

        ## define the LSTM, dropout and fully connected layers
        self.lstm = nn.LSTM(
            len(self.unique_tokens),
            n_hidden,
            n_layers,
            dropout=drop_prob,
            batch_first=True,
        )
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden, len(self.unique_tokens))

    def forward(
        self, x: torch.Tensor, hidden: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        r_output, hidden = self.lstm(x, hidden)
        out = self.dropout(r_output)
        # Stack up LSTM outputs using view. You may need to use contiguous to reshape the output.
        out = out.contiguous().view(-1, self.n_hidden)
        ## Get the output for classification.
        out = self.fc(out)
        return out, hidden
# инициализаци скрытого слоя
    def init_hidden(
        self, batch_size: int, weight_device: torch.device
    ) -> Tuple[torch.Tensor]:
        """
        Creates two new zero tensors for hidden state and cell state of LSTM
        :param batch_size: number of sequences per batch
        :param weight_device: torch.device("cuda") for GPU init or torch.device("cpu") for CPU init
        :return: tuple of two tensors of shape [n_layers x batch_size x n_hidden]
        """
        weight = next(self.parameters()).data
        hidden = (
            weight.new(self.n_layers, batch_size, self.n_hidden)
            .zero_()
            .to(weight_device),
            weight.new(self.n_layers, batch_size, self.n_hidden)
            .zero_()
            .to(weight_device),
        )
        return hidden

In [ ]:
class CharRNNModule(pl.LightningModule): # тренировка модели
    def __init__(
        self,
        unique_tokens: Tuple[str],
        n_hidden: int = 1024,
        n_layers: int = 2,
        drop_prob: float = 0.5,
        batch_size: int = 128, # поделим на данные батчи и пакетами подадим на вход модели
        seq_length=256,
        lr: float = 0.001,
    ) -> None:
        super().__init__()
        self.model = CharRNN(unique_tokens, n_hidden, n_layers, drop_prob)
        self.hidden = None
        self.loss = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.n_chars = len(unique_tokens) # кол-во символов = уникальные токены
        self.batch_size = batch_size
        self.seq_length = seq_length

    def training_step( # шаг тренировки
        self, train_batch: Tuple[torch.Tensor, torch.Tensor]
    ) -> torch.Tensor:
        x, y = train_batch # подаем на вход тренировачный батч
        x, y = x.squeeze(0), y.squeeze(0) # делаем кодирование x,y
        x = one_hot_encode(x, self.n_chars)

        if self.hidden is None:
            self.hidden = self.model.init_hidden(self.batch_size, self.device)
        self.hidden = tuple([each.data for each in self.hidden])

        output, self.hidden = self.model(x, self.hidden)
        loss = self.loss(output, y.reshape(self.batch_size * self.seq_length).long())
# обнуление весов
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return self.optimizer

## 3. Обучение модели

По классике, используем оптимизатор Adam и кросс-энтропию. Но без пары особенностей не обойтись:
* Во время цикла будем отделять скрытое состояние от его истории, потому что скрытое состояние LSTM является кортежем скрытых состояний.
* Будем использовать gradient clipping, чтобы избавиться от взрывающихся градиентов.

In [ ]:
# data
train_dataset = AnnaData(encoded, batch_size=128, seq_length=256)
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=1,  # batching is already implemented on our side
    shuffle=False,
    num_workers=1,  # don't change: it will lead to the wrong implementation
)
# model
char_rnn = CharRNNModule(unique_chars, n_hidden=1024, batch_size=128)
# trainer
trainer = pl.Trainer(max_epochs=15, gradient_clip_val=1.0, accelerator="gpu", devices=1)
trainer.fit(char_rnn, train_dataloaders=train_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params | Mode 
---------------------------------------------------
0 | model | CharRNN          | 13.0 M | train
1 | loss  | CrossEntropyLoss | 0      | train
---------------------------------------------------
13.0 M    Trainable params
0         Non-trainable params
13.0 M    Total params
52.097    Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=15` reached.


## 4. Применение модели

Сперва сохраним обученную модель, чтобы можно было загрузить её позже. В следующей ячейке сохраняются параметры, необходимые для создания той же архитектуры, гиперпараметры скрытого слоя и токены.

Сохраняем веса: /content/rnn_x_epoch.net

Можно открыть только torch, что-то нечитаемое.

In [ ]:
net = char_rnn.model # как выбрать слово или букву, которая будет следующей? (картинка, где нужно выбрать одно число)
checkpoint = {
    "n_hidden": net.n_hidden,
    "n_layers": net.n_layers,
    "state_dict": net.state_dict(),
    "tokens": net.unique_tokens,
}

with open("rnn_x_epoch.net", "wb") as f:
    torch.save(checkpoint, f)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

### Делаем предсказания (Как выбирать следующий символ?)

Сгенерируем текст! Для предсказания продолжения текста мы передаём в сеть последний символ, она предсказывает следующий символ, который мы снова передаем на вход, получаем ещё один предсказанный символ и так далее. Наши прогнозы основаны на категориальном распределении вероятностей по всем возможным символам. Мы можем ограничить число символов на каждом шаге генерации, чтобы сделать получаемый предсказанный текст более разумным, рассматривая только некоторые, наиболее вероятные символы. С одной стороны, такой подход позволит нам рассматривать не только самую вероятную последовательность с точки зрения прогноза модели. С другой стороны, мы будем работать с ограниченным набором сгенерированных вариантов, поэтому избавимся от совсем уж шумовых прогнозов. Узнать больше можно [здесь](https://pytorch.org/docs/stable/generated/torch.topk.html#torch.topk).

In [ ]:
def predict_next_char(
    model: torch.nn.Module, char: str, h: torch.Tensor = None, top_k: int = None
) -> Tuple[str, torch.Tensor]:
    """
    Given a character and a model, predicts next character in the sequence
    :param model: model that outputs next token probability distribution
    :param char: last character of the sequence to continue generation from
    :param h: hidden state of the model
    :param top_k: number of most probable tokens to be chosen from
    :return: tuple of next character and new hidden state
    """
    # tensor inputs
    x = torch.tensor([[model.char2int[char]]])
    x = one_hot_encode(x, len(model.unique_tokens))
    x = x.to(device)

    h = tuple([each.data for each in h])

    out, h = model(x, h)

    # get the character probabilities
    p = torch.nn.functional.softmax(out, dim=1).data.cpu()

    # get top characters (выбираем одну вероятность и узнаем индекс - сортируем число)
    if top_k is None:
        top_ch = torch.arange(len(model.unique_tokens))
    else:
        p, top_ch = p.topk(top_k)
# мы перевели вектор с весами (посчитали веса)
    p.squeeze_()
    top_ch.squeeze_()
    char = top_ch[torch.multinomial(p / p.sum(), 1)]

    return model.int2char[char.item()], h

### Priming и генерирование текста

Нужно задать скрытое состояние, чтобы сеть не генерировала произвольные символы.

In [ ]:
def sample(model, size, prime="The", top_k=None):
    model.to(device)
    model.eval()

    # run through the prime characters
    chars = [ch for ch in prime]
    h = model.init_hidden(1, device)
    for ch in prime:
        char, h = predict_next_char(model, ch, h, top_k=top_k)

    chars.append(char)

    # pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict_next_char(model, chars[-1], h, top_k=top_k)
        chars.append(char)

    return "".join(chars)

In [ ]:
print(sample(net, 1000, prime="Anna", top_k=5))

Anna;,?!, 
 he had to stain that they were to sen aboniting him and tame talking the chown and
any happiness
and hus sechant, thaights wish of his find this who had tere all at his hand, too were heard a darent tamily befter the carrea seated and worted the
manse and as it importent out and alons, stirn of all the mire, they'rist the man to the did not tell him with with her to so how that the plans than she was the madsely and this talking her husband what he't the werring with all with his
first of her. Alexey Alexandrovitch streed to be meaning he was the crows were, and the countest went in the
rowing trought to ask
her, he was believed the misses, but
she had stranged to the sare the parys with strick in
the count and antere that the plancial serst and with his
stats of was the some, the came," and the
ridse as he
had been a position is a sumperiof wanting to
the sorts to that he had train in the camily
to all with the stopt was
a marred
to the stepsed and at overing that he down 

В некоторых местах соблюдена грамматика, но есть лексические ошибки, несуществующие слова.

Предсказание следующего символа без учета порядка.

### Загрузка чекпоинта и генерация

In [ ]:
with open("rnn_x_epoch.net", "rb") as f: # просто выгрузили модель /content/rnn_x_epoch.net
    checkpoint = torch.load(f)

loaded = CharRNN(
    checkpoint["tokens"],
    n_hidden=checkpoint["n_hidden"],
    n_layers=checkpoint["n_layers"],
)
loaded.load_state_dict(checkpoint["state_dict"])

# sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said to be mentering. The senvest of the some
to her away for. His see of the cormorday. He sould her faller at her
she was not sometered to
saw to him to see, the sace offenent to the
strong, and see her hands at one at the from thome to be talking a lently whom he could no take him with a crack and and was the stroagh on her subjection to her hand of a letter trat, and a such a far and was
a marry out of the cornors, the
shound with the friend with talken to him on the same that the sume one of a senger in still to him, we will see her from this thing in the
parss of the point on her find of him and should now think
whene an inseared a low, and he could not time that the proming
with harss were time to him,
with a could not though the compost of a countess he did not belied whom he had been too would all all the
seming of
the meading,
and he had not
to stay, but the who would stend her wife, and to begond
the true of the plassion of the
commors, and him to seak to her and
s